In [21]:
import sys

# # 1. Retrieve the file name passed as an argument
# file_name = "school_name_01.csv"  # The first argument should be the file name


# 2. Construct the full file path (use f-string to properly include the variable)
file_path = f"Files/student_data/{file_name}"# Assuming the following structure in OneLake:
#file_path = "Files/student_data/school_name_01.csv"

# 3. Load the CSV file into a Spark DataFrame
df = spark.read.csv(file_path, header=True, inferSchema=True)

# 4. Show the DataFrame or perform your transformations
df.show()



StatementMeta(, 2ad807b6-61d6-4ecb-a911-932ec991ec3f, 23, Finished, Available, Finished)

+-----------+----------+---------+---------+---------+
|school_name|  std_name|subject_1|subject_2|subject_3|
+-----------+----------+---------+---------+---------+
| 1st school|student 01|       50|       20|       50|
| 1st school|student 02|       60|       90|       80|
| 1st school|student 03|       70|       80|       90|
| 1st school|student 04|       20|       10|       30|
| 1st school|student 05|       44|       33|       55|
+-----------+----------+---------+---------+---------+



In [27]:
from pyspark.sql.functions import col, when
# 4. Add a 'pass_mark' column for subject_1_status
#    Logic: If subject_1 > 35, mark as 'pass', otherwise 'fail'
df_transformed = df.withColumn(
    "subject_1_status",
    when(col("subject_1") > 35, "pass").otherwise("fail") 
).withColumn(
    "subject_2_status",
    when(col("subject_2") > 35, "pass").otherwise("fail") 
).withColumn(
    "subject_3_status",
    when(col("subject_2") > 35, "pass").otherwise("fail") 
)
# 5. Show the transformed DataFrame
df_transformed.show()

StatementMeta(, 2ad807b6-61d6-4ecb-a911-932ec991ec3f, 29, Finished, Available, Finished)

+-----------+----------+---------+---------+---------+----------------+----------------+----------------+
|school_name|  std_name|subject_1|subject_2|subject_3|subject_1_status|subject_2_status|subject_3_status|
+-----------+----------+---------+---------+---------+----------------+----------------+----------------+
| 1st school|student 01|       50|       20|       50|            pass|            fail|            fail|
| 1st school|student 02|       60|       90|       80|            pass|            pass|            pass|
| 1st school|student 03|       70|       80|       90|            pass|            pass|            pass|
| 1st school|student 04|       20|       10|       30|            fail|            fail|            fail|
| 1st school|student 05|       44|       33|       55|            pass|            fail|            fail|
+-----------+----------+---------+---------+---------+----------------+----------------+----------------+



In [ ]:
from delta.tables import DeltaTable

# Write the transformed data into the Silver layer
df_transformed.write.option("overwriteSchema", "true").format("delta").mode("overwrite").save("Tables/student_data")